# 앙상블 학습 ( Ensemble )

* 여러 개의 Classifier 를 생성하고 그 예측을 결합함으로써 보다 정확한 최종 예측을 도출하는 기법 
* 정형 데이터 분류 시 가장 뛰어난 성능<br><br>
* 앙상블 학습 유형<br>
    * 보팅 (Voting) & 배깅 (Bagging)
         * 여러 개의 분류기가 투표를 통해 최종 예측 결과 결정
         * 보팅 
             * 서로 다른 알고리즘을 가진 분류기 (Classifier) 를 결합하는 것 
         * 배깅 (Bagging)
             * 같은 알고리즘을 가진 개별 분류기에게 전체 데이터에서 일부만 중첩되게 데이터 세트를 샘플링해서 추출하는 방식 (Bootstrapping) 으로 샘플링된 데이터 세트에 대해서 학습을 통해 개별적인 예측을 수행한 결과를 보팅을 통해 최종 예측 결과를 선정하는 방식 [참고](https://media.vlpt.us/images/kjpark4321/post/c5208df8-0c2e-48b6-a727-b8996633d167/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7%202021-01-14%20%EC%98%A4%EC%A0%84%201.02.01.png)
             * Random Forest <br><br>
     
    * 부스팅 (Boosting)
        * 여러 개의 분류기가 순차적으로 학습을 수행하되, 앞에서 학습한 분류기가 예측이 틀린 데이터에 대해서는 올바르게 예측 할 수 있도록 다음 분류기에게는 가중치를 부여하면서 학습과 예측을 진행하는 것 
        * GBM, XGBoost, LightGBM, Stacking
         
        
        

## 보팅 (Voting)

* Hard Voting : 다수의 classifier의 예측 레이블 값을 __다수결__로 최종 값으로 결정  
* Soft Voting : 다수의 classifier의 레이블 값 결정 확률을 __평균__하여 이들 중 확률이 가장 높은 레이블 값을 최종 값으로 선정 <br>

[이미지 참고](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcvYig2%2FbtqKsojGyfl%2FiKazMxfc8GKeWa1OHYjHH0%2Fimg.png), class = label<br><br>
사이킷런은 Voting 방식의 앙상블 구현을 위해 VotingClassifier 클래스를 제공한다.<br>
내장 dataset, 위스콘신 유방암 데이터 load_breast_cancer() 를 이용해서 예측 분석을 해보자.<br>
* 보팅 Classifier 는 LogisticRegression 과 KNN 을 기반으로 구현한다.

In [1]:
import pandas as pd 
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer = load_breast_cancer()

cancer_df = pd.DataFrame(cancer.data,columns=cancer.feature_names)

cancer_df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [2]:
lr_clf = LogisticRegression(max_iter=5000)
knn_clf = KNeighborsClassifier()

# 생성 인자 
# estimator : 리스트 값으로 보팅에 사용 될 Classifier 객체들을 튜플 형식으로
# voting : 하드 or 소프트 결정 (default = hard)
vo_clf = VotingClassifier(estimators=[('LR',lr_clf),('KNN',knn_clf)],voting='soft')

X_train , X_test, y_train,y_test = train_test_split(cancer.data, cancer.target,test_size=0.2, random_state= 501)

# Voting 학습 / 예측 / 평가 
vo_clf.fit(X_train,y_train)

pred = vo_clf.predict(X_test)

print('Voting 분류기 정확도 : {0:.4f}'.format(accuracy_score(y_test,pred)))

# Voting 내에 있던 개별 classifier 의 학습 / 예측 / 평가 
classifiers = [lr_clf,knn_clf]
for classifier in classifiers:
    classifier.fit(X_train,y_train)
    each_pred = classifier.predict(X_test)
    class_name = classifier.__class__.__name__
    print('{0} 정확도 :{1:.4f}'.format(class_name,accuracy_score(y_test,each_pred)))

Voting 분류기 정확도 : 0.9561
LogisticRegression 정확도 :0.9474
KNeighborsClassifier 정확도 :0.9474


여러 개의 기반 분류기를 결합한 Voting 의 예측 정확도가 개별 분류기보다 약간 높게 측정되었다. <br>
단, Voting 이라 해서 무조건 예측 성능이 향상 되는 것은 아니다. <br>
단일 ML 알고리즘은 고정된 관점을 가져 유연성이 떨어 질 수 있기 때문에 앙상블이 보다 뛰어난 예측 성능을 가지는 경우가 많을 뿐이다.<br><br>
+. Voting 과 Stacking 은 서로 다른 알고리즘을 기반으로 하고 있지만, Bagging 과 Boosting 은 대부분 결정 트리 알고리즘을 기반으로 한다.<br>
결정 트리 알고리즘은 쉽고 직관적이지만, 과적합 문제가 있다 했었고 앙상블 학습을 통해 수백개의 많은 분류기를 결합해서 이 문제를 극복할 수 있다.

## Random Forest 

* 배깅의 대표적인 알고리즘 <br><br>
* Random Forest 는 전체 데이터에서 Bootstrapping 방식으로 각 각의 데이터로 샘플링해 여러 개의 개별 결정 트리 classifier 에 학습을 시킨 뒤 최종적으로 모든 classifier 가 voting 을 통해 예측 결정을 한다. [참고](https://media.vlpt.us/images/kjpark4321/post/f0f5c31d-c4c9-4e2a-8bb4-53548aad5b77/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7%202021-01-14%20%EC%98%A4%EC%A0%84%201.53.55.png)<br><br>
* 부가 설명<br><br>
    * Bootstrapping 분할 방식 : 개별 트리가 학습 할 dataset = subset 으로 분리 하기 위해 전체 데이터에서 일부가 중첩되게 샘플링 하는 것<br><br>  
    * classifier 로 대부분 DecisionTree 사용 
    * 랜덤 포레스트의 subset 은 bootstrapping 으로 데이터가 임의로 만들어진다. [참고](https://media.vlpt.us/images/kjpark4321/post/5c47476a-7abf-45b8-a5ba-b442305933d8/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7%202021-01-14%20%EC%98%A4%EC%A0%84%201.57.25.png)
    * n_estimators 속성으로 분할 될 subset 갯수 지정 
    * 사이킷런은 RandomForestClassifier 클래스를 통해 Random Forest 지원 

이 처럼 데이터가 중첩된 개별 서브 세트에 decisiontree classifier 를 각 각 적용하는 것이 Random Forest<br><br>
human_activity_recognition.ipynb 파일의 get_human_activity_recognition_dataset 사용자 정의 함수 가지고 와서 Random Forest 를 사용해보자.

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),columns=['dup_cnt']) 
    feature_dup_df = feature_dup_df.reset_index() # column name 으로 되어 있는 index reset 
    
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(),feature_dup_df,how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name','dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1])
                                                                                             if x[1]>0 else x[0],axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'],axis=1)
    return new_feature_name_df

def get_human_activity_recognition_dataset():
    feature_name_df = pd.read_csv("UCI_HAR_Dataset/features.txt",sep='\s+',header=None, names=['column_index','column_name'])
    # 컬럼명 중복 처리
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # 전처리된 feature 들을 학습, 테스트 dataset 의 feature 로 넣어줘야 해서 list 로 변환
    feature_name = new_feature_name_df.iloc[:,1].values.tolist()
    
    X_train = pd.read_csv("UCI_HAR_Dataset/train/X_train.txt",sep='\s+',header=None, names=feature_name)
    X_test = pd.read_csv("UCI_HAR_Dataset/test/X_test.txt",sep='\s+',header=None, names=feature_name)
    
    y_train = pd.read_csv("UCI_HAR_Dataset/train/y_train.txt",sep='\s+',header=None, names=['action'])
    y_test = pd.read_csv("UCI_HAR_Dataset/test/y_test.txt",sep='\s+',header=None, names=['action'])
    
    return X_train , X_test, y_train, y_test

X_train, X_test, y_train , y_test = get_human_activity_recognition_dataset()

# Random Forest 학습 및 예측 성능 평가 
rf_clf = RandomForestClassifier(random_state=110)
rf_clf.fit(X_train,y_train)
rf_pred = rf_clf.predict(X_test)
print('Random Forest 정확도: {0:.4f}'.format(accuracy_score(y_test,rf_pred)))

Random Forest 정확도: 0.9264


### Random Forest 하이퍼 파라미터 및 튜닝

트리 기반의 앙상블 알고리즘은 하이퍼 파라미터가 너무 많아, 튜닝에 소모 되는 시간이 많음에도 예측 성능이 크게 향상되는 경우가 많지 않다는 아쉬움이 있다. 일반적으로 Random Forest 는 DecisionTree 에서 사용되는 하이퍼 파라미터와 동일한 경우가 대부분 이다. 
* n_estimators : Random Forest 에서 DecisionTree 의 갯수 지정 . default = 10 개 . 많이 설정 할 수록 좋은 성능을 기대 할 수 있지만 그만큼 수행 시간이 오래 걸리기 때문에 유의


* 그 외는 DecisionTree 하이퍼 파라미터와 동일 

늘 그랬듯이 하이퍼 파라미터 튜닝은 GridSearchCV 를 이용한다


In [6]:
from sklearn.model_selection import GridSearchCV

params = {'n_estimators':[100],
         'max_depth':[6,8,10,12],
         'min_samples_leaf':[8,12,18],
         'min_samples_split':[8,16,20]
         }

rf_clf = RandomForestClassifier(random_state=11,n_jobs=-1)
grid_cv = GridSearchCV(rf_clf,param_grid=params,cv=2,n_jobs=-1)
grid_cv.fit(X_train,y_train)

print('최적 하이퍼 파라미터: ',grid_cv.best_params_)
print('최고 정확도: {0:.4f}'.format(grid_cv.best_score_),'\n\n')

estimator = grid_cv.best_estimator_

pred = estimator.predict(X_test)

print('테스트 데이터 세트 예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

최적 하이퍼 파라미터:  {'max_depth': 12, 'min_samples_leaf': 12, 'min_samples_split': 8, 'n_estimators': 100}
최고 정확도: 0.9144 


테스트 데이터 세트 예측 정확도: 0.9250


### 중요한 피처는? 

In [29]:
# iplot
import plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

randomforest_important_feature_df = pd.DataFrame(randomforest_feature,index=X_train.columns)
randomforest_important_feature_df.columns = ['feature_importance']
randomforest_important_feature_df.sort_values(by='feature_importance',ascending=False,inplace=True)
randomforest_important_feature_df[:20]

randomforest_important_feature_df[:20].iplot(kind='barh')
